In [19]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import MinMaxScaler #scikit-learn==0.21.3
from xgboost import XGBClassifier #xgboost==0.90
from sklearn.manifold import TSNE
import xgboost
from sklearn.preprocessing import LabelEncoder

In [20]:
xgboost.__version__

'0.90'

## G1 predictions

In [21]:
data = pd.read_csv('../Field_training_data_justMMETSP.csv')
labels = pd.read_csv('../Field_training_labels_justMMETSP.csv')
data = data.reset_index(drop=True)
labels = labels.reset_index(drop=True)

idx = labels.index[labels['Trophic mode'] == 'Un']
train_labels = labels.drop(idx)
train_data = data.drop(idx)

features = pd.read_csv('../Extracted_Pfams.csv')

G1_TPM_merged_lats = pd.read_csv('../../g1Surface/G1_surface_tpm_updatedMarferret_marmicroDb2023_noOutliers_fixedTPM_fall2023.csv')

In [22]:
G1_TPM_merged_lats.columns[(G1_TPM_merged_lats == 0).all()]

Index(['PF14259'], dtype='object')

In [23]:
features = features['Pfam'].str.split('.').str[0]

In [24]:
len(features)

1046

In [25]:
original_features = set(G1_TPM_merged_lats.columns)

In [26]:
missing_features = set(features) - original_features
missing_features

{'PF02577',
 'PF03986',
 'PF04413',
 'PF08017',
 'PF08217',
 'PF08802',
 'PF10102',
 'PF13427'}

In [27]:
#Add the missing features to G1_TPM_merged_lats, filled with 0 values
for feature in missing_features:
    G1_TPM_merged_lats[feature] = 0

In [28]:
G1_TPM_merged_lats = G1_TPM_merged_lats[features]

In [29]:
le = LabelEncoder()

In [30]:
train_data = train_data[features]

In [31]:
from sklearn.model_selection import train_test_split
from sklearn import svm

In [32]:
X,y = train_data, le.fit_transform(train_labels['Trophic mode'])
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [33]:
G1 = scaler.transform(G1_TPM_merged_lats)

In [34]:
model = XGBClassifier(gamma=0.0, learning_rate=0.5, max_depth=3, n_estimators=10, reg_lambda=0.)
model.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.0,
              learning_rate=0.5, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=10, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=0.0, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [35]:
xg_predictions_G1 = model.predict(G1)

In [36]:
prob = model.predict_proba(G1)

In [37]:
probDF = pd.DataFrame(data={'probability':np.max(prob, axis = 1)})

In [38]:
G1_predictions = pd.DataFrame(data={'xg_pred':xg_predictions_G1})

In [39]:
G1_predictions.to_csv('../../g1Surface/G1_trophicModePredictions_surface_updatedMarferret_marmicroDb2023_noOutliers_fall2023_lambert',index=False)

In [40]:
probDF.to_csv('../../g1Surface/G1_trophicModePredictionsProbabilities_surface_updatedMarferret_marmicroDb2023_noOutliers_fall2023_lambert',index=False)